In [2]:
import os
import re
import subprocess
from datetime import datetime
from pycountry import countries

In [39]:
def parse(cleaned_list: list):
    if (len(cleaned_list[0]) == 44) or (len(cleaned_list[0]) == 36):
        # Extracting the first letter of the first element
        Passport_type = cleaned_list[0][0]  # Accessing the second character
        if cleaned_list[0][1] == '<':
            Passport_subtype = None
        else:
            # Extracting the second letter of the first element
            Passport_subtype = cleaned_list[0][1]  # Accessing the second character
        Issuing_country = cleaned_list[0][2:5]
        remains = cleaned_list[0][5:] #Comntains name and surname for further processing
        Surname = remains.split('<<', 1)[0].replace('<', ' ')
        Name = remains.split('<<', 1)[1].strip('<').replace('<', ' ')
        Passport_number = cleaned_list[1][0:9]
        Check_digit1 = cleaned_list[1][9]
        Nationality = cleaned_list[1][10:13]
        Birth_Date = cleaned_list[1][13:19]
        Check_digit2 = cleaned_list[1][19]
        if cleaned_list[1][20] == '<':
            Gender = 'unspecified'
        else:
            Gender = cleaned_list[1][20]
        Expiration_date = cleaned_list[1][21:27]
        Check_digit3 = cleaned_list[1][27]
        if len(cleaned_list[0]) == 44:
            Personal_number = cleaned_list[1][28:42].strip('<')
            Check_digit4 = cleaned_list[1][42]
            Check_digit5 = cleaned_list[1][43]
        elif len(cleaned_list[0]) == 36:
            Personal_number = cleaned_list[1][28:35].strip('<')
            Check_digit4 = cleaned_list[1][35]
    details = [Name, Surname, Passport_type, Passport_subtype, Issuing_country, Passport_number, Nationality, Birth_Date, Gender, Expiration_date, Personal_number, Check_digit1, Check_digit2, Check_digit3, Check_digit4, Check_digit5]
    details = [element.strip("<") for element in details]
    return details

In [47]:

def ExtractData(filepath):
    base_workdir = os.getcwd()
    try:
        # Run the Node.js command with 'cwd' set to the target directory
        node_command = ["node", "run/getMrz.js", "--file", filepath]
        result = subprocess.run(node_command, cwd=base_workdir, check=True)
        # Print any error output if present
        if result.stderr:
            logging.exception("Error output:")
            logging.exception(result.stderr)
    except subprocess.CalledProcessError as e:
        logging.exception("Error running the command:", e)
    
    # Get the file name from the file_location variable
    file_name = os.path.basename(filepath)

    # Split the file name and extension
    file_name, _ = os.path.splitext(file_name)

    # Change the file extension to .png
    file_name = file_name + ".png"

    # Access the /out/crop subdirectory
    new_dir = os.path.join(os.path.dirname(filepath), "out", "crop")

    # Join the new directory with the new file name
    filepath = os.path.join(new_dir, file_name)
    
    try:
        # Run the Node.js command with 'cwd' set to the target directory
        node_command = ["node", "run/readMrz.js", "--file", filepath]
        result = subprocess.run(node_command, cwd=base_workdir, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        print("Error running the command:", e)

    result_stdout = result.stdout # Updating variable name prevents potential problems with "."
    
    # Find the index of the first occurrence of 'mrz:'
    start_mrz_index = result_stdout.find('mrz:')

    if start_mrz_index != -1:
        # Find the index of the first '[' after 'mrz:'
        start_bracket_index = result_stdout.find('[', start_mrz_index)

        # Find the index of the first ']' after 'mrz:'
        end_bracket_index = result_stdout.find(']', start_mrz_index)

        if start_bracket_index != -1 and end_bracket_index != -1:
            # Extract the substring between the first '[' and the first ']' after 'mrz:'
            result = result_stdout[start_bracket_index + 1: end_bracket_index]

        else:
            print("No matching brackets found after 'mrz:'")
    else:
        print("No 'mrz:' found in the text.")
    
    # Use regular expressions to remove the escape sequences for colors and leading/trailing whitespaces
    cleaned_result = re.sub(r'\x1b\[\d+m', '', result).strip()

    # Split the cleaned result into a list using splitlines() and then split by comma
    result_list = cleaned_result.split(',')

    # Clean up the elements by removing single quotes and extra spaces, and create a new list
    cleaned_list = [element.strip().strip("'") for element in result_list]
    return cleaned_list

#Testing
tester = ExtractData('C:\\Users\\iyedn\OneDrive\\Desktop\\stage\\mrz-detection-master\\img\\Specimen_Personal_Information_Page_South_Korean_Passport.jpg')
tester

['PMK0RLEE<<SUYE0N<<<<<<<<<<<<<<<<<<<<<<<<<<<<',
 'M708B90985KOR8507022F24041522154710V17627B84']

In [48]:
parse(tester)

['SUYE0N',
 'LEE',
 'P',
 'M',
 'K0R',
 'M708B9098',
 'KOR',
 '850702',
 'F',
 '240415',
 '2154710V17627B',
 '5',
 '2',
 '2',
 '8',
 '4']